In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import vdmlab as vdm

In [ ]:
linear = dict()
linear['position'] = np.concatenate((np.linspace(0, 10, 10), np.linspace(10, 0, 10), np.linspace(0, 10, 20)))
linear['time'] = np.linspace(0, 100, 40)

In [ ]:
plt.plot(linear['time'], linear['position'])
plt.show()

In [ ]:
import random
random.seed(0)
spikes = dict(time=[[], [], [], []])
for i in range(10):
    spikes['time'][0].append(random.uniform(0, 100))
for i in range(15):
    spikes['time'][1].append(random.uniform(20, 30))
for i in range(15):
    spikes['time'][2].append(random.uniform(60, 80))   
for i in range(15):
    spikes['time'][3].append(random.uniform(5, 10))
for i in range(5):
    for neuron_spikes in spikes['time']:
        neuron_spikes.append(random.uniform(0, 100))

In [ ]:
for idx, neuron_spikes in enumerate(spikes['time']):
    plt.plot(neuron_spikes, np.ones(len(neuron_spikes))+idx, '|', color='k', ms=10, mew=1)
plt.ylim(0.5, 4.5)
plt.show()

In [ ]:
tuning = vdm.tuning_curve(linear, spikes['time'], num_bins=50)

In [ ]:
def tuning_curve(position_z, spike_times, num_bins=100, sampling_rate=1/30.0, filter_type='gaussian', gaussian_std=3):
    """ Computes tuning curves for neurons relative to linear position.

    Parameters
    ----------
    position_z : dict
        With position, time as keys
    spike_times : list of lists
        Where each inner list contains the spike times for an
        individual neuron.
    num_bins : int
        Defaults to 100 if not specified
    sampling_rate : float
        Defaults to 1/30. if not specified.
    filter_type : str, optional
        Defaults to 'gaussian' to smooth with a gaussian filter.
        No smoothing if None.
    gaussian_std : int
        Defaults to 18.

    Returns
    -------
    out_tc : list of lists
        Where each inner list contains the tuning curves for an
        individual neuron.

    Notes
    -----
    Input position_z and spike_times should be from the same time
    period. Eg. when the animal is running on the track.

    """
    linear_start = np.min(position_z['position'])
    linear_stop = np.max(position_z['position'])
    bin_edges = np.linspace(linear_start, linear_stop, num=num_bins)
    bin_centers = np.array((bin_edges[1:] + bin_edges[:-1]) / 2.)
    tc = []
    occupancy = np.zeros(len(bin_centers))
    for position in position_z['position']:
        pos_idx = find_nearest_idx(bin_centers, position)
        occupancy[pos_idx] += sampling_rate
    occupied_idx = occupancy > 0
    for neuron_spikes in spike_times:
        spike_z = np.zeros(len(bin_centers))
        for spike_time in neuron_spikes:
            bin_idx = find_nearest_idx(position_z['time'], spike_time)
            which_bin = find_nearest_idx(bin_centers, position_z['position'][bin_idx])
            spike_z[which_bin] += 1
        firing_rate = np.zeros(len(bin_centers))
        firing_rate[occupied_idx] = spike_z[occupied_idx] / occupancy[occupied_idx]
        tc.append(firing_rate)

    if filter_type == 'gaussian':
        filter_multiplier = 6
        out_tc = []
        gaussian_filter = signal.get_window(('gaussian', gaussian_std), gaussian_std*filter_multiplier)
        normalized_gaussian = gaussian_filter / np.sum(gaussian_filter)
        for firing_rate in tc:
            out_tc.append(np.convolve(firing_rate, normalized_gaussian, mode='same'))
    else:
        print('Tuning curve with no filter.')
        out_tc = tc

    return out_tc

In [ ]:
colours = ['b', 'g', 'r', 'c']

x = list(range(0, np.shape(tuning)[1]))
for i, neuron_tc in enumerate(tuning):
    plt.plot(neuron_tc, color=colours[i])
#     plt.fill_between(x, 0, neuron_tc, facecolor=colours[i])
plt.show()

In [ ]:
np.shape(tuning)[1]